In [4]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
mp_key = "zwe7ZzatCkKkvB6PUpwVmS0nGdBcJr1R"
mp_id = "mp-1569"

In [6]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Structure symmetry

## Première opération de symétrie

In [24]:
i_atom = 1
i_symmop = 2

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [18]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [19]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.25 0.25 0.75]


In [20]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[-0.25 -0.25 -0.75]


In [16]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

8 [0.25 0.25 0.25]


La première opération $\begin{pmatrix}-x&-y&-z\end{pmatrix}$ est définie par une matrice de rotation <div>$\begin{pmatrix}-1&0&0\\0&-1&0\\0&0&-1\end{pmatrix}$ et une translation nulle.</div> 
    
Cette symétrie est une inversion de centre $\begin{pmatrix}0&0&0\end{pmatrix}$.    

Cette symétrie s'opère sur l'atome 1, atome de Be, qui donne un point en dehors de la maille, donc l'atome est renvoyé dans la maille par translation à l'atome de Be numéro 3.

Elle s'opère sur l'atome Be situé en $\begin{pmatrix}0.25&0.25&0.75\end{pmatrix}$, l'atome issu de l'opération se retrouve en $\begin{pmatrix}-0.25&-0.25&-0.75\end{pmatrix}$.

## Deuxième opération de symétrie

In [82]:
i_atom = 2
i_symmop = 15

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [83]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [84]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.75 0.75 0.25]


In [85]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[ 0.75  0.75 -0.25]


In [86]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

7 [0.75 0.75 0.75]


La deuxième opération $\begin{pmatrix}y&x&-z\end{pmatrix}$ est définie par une matrice de rotation <div>$\begin{pmatrix}0&1&0\\1&0&0\\0&0&-1\end{pmatrix}$ et une translation nulle.</div>
    
Il s'agit d'une rotation d'ordre 2 autour de l'axe $\begin{bmatrix}1&1&0\end{bmatrix}$.  
    
Elle s'opère sur l'atome C situé en $\begin{pmatrix}0.75&0.75&0.25\end{pmatrix}$, l'atome issu de l'opération se retrouve en $\begin{pmatrix}0.75&0.75&-0.25\end{pmatrix}$.

## Troisième opération de symétrie

In [118]:
i_atom = 6
i_symmop = 7

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [119]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]


In [120]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.25 0.75 0.75]


In [121]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[ 0.75 -0.25  0.75]


In [122]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

7 [0.75 0.75 0.75]


La deuxième opération $\begin{pmatrix}y&-x&z\end{pmatrix}$ est définie par une matrice de rotation <div>$\begin{pmatrix}0&1&0\\-1&0&0\\0&0&1\end{pmatrix}$ et une translation nulle.</div>
    
Il s'agit d'une rotation d'ordre 2 autour de l'axe $\begin{bmatrix}0&1&1\end{bmatrix}$.  
    
Elle s'opère sur l'atome Be situé en $\begin{pmatrix}0.25&0.75&0.75\end{pmatrix}$, l'atome issu de l'opération se retrouve en $\begin{pmatrix}0.75&-0.25&0.75\end{pmatrix}$.

Cette symétrie s'opère sur l'atome 6, atome de Be, qui donne un point en dehors de la maille, donc l'atome est renvoyé dans la maille par translation à l'atome de Be numéro 4.